In [2]:
import pandas as pd
import plotly.express as px

/opt/anaconda3/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning:

IPython.utils.traitlets has moved to a top-level traitlets package.



# GTFS exploration

In [10]:
class read_gtfs(object):
    """a class to read gtfs object
       """
    def __init__(self, update_date):
        self.agency=pd.read_csv('data/'+update_date+'/agency.txt', sep=",")
        self.calen_date=pd.read_csv('data/'+update_date+'/calendar_dates.txt', sep=",")
        self.calendar=pd.read_csv('data/'+update_date+'/calendar.txt', sep=",")
        self.rout=pd.read_csv('data/'+update_date+'/routes.txt', sep=",")
        self.shape=pd.read_csv('data/'+update_date+'/shapes.txt', sep=",")
        self.stop_time=pd.read_csv('data/'+update_date+'/stop_times.txt', sep=",")
        self.stop=pd.read_csv('data/'+update_date+'/stops.txt', sep=",")
        self.trip=pd.read_csv('data/'+update_date+'/trips.txt', sep=",")


In [11]:
latest_gtfs=read_gtfs('2020-5-29')
gtfs_19_9=read_gtfs('2019-9-27')



In [59]:
#show number of trip
print('2019-9-17: trip number',gtfs_19_9.trip.shape[0])
print('2020-5-29: trip number',latest_gtfs.trip.shape[0])




2019-9-17: trip number 9232
2020-5-29: trip number 7349


In [16]:
latest_gtfs.trip.service_id.nunique()

8

In [61]:
#show number of route
print('2019-9-17: route number',gtfs_19_9.rout.shape[0])
print('2020-5-29: route number',latest_gtfs.rout.shape[0])


2019-9-17: route number 76
2020-5-29: route number 45


In [87]:
df=gtfs_19_9.stop_time
df1=df[df['trip_id']==180166]
df2=df1.merge(gtfs_19_9.stop,on='stop_id',how='left')
df2.head()
lat_center=df2['stop_lat'].mean()
lon_center=df2['stop_lon'].mean()

In [92]:
#show trip 180166

fig = px.line_mapbox(df2, lat="stop_lat", lon="stop_lon",  zoom=3, height=300)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=10, mapbox_center_lat = lat_center,mapbox_center_lon = lon_center,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

# Check Ridership

In [4]:
rider=pd.read_excel('data/Boarding_Alighting_Sample_Data_w_load.xlsx')


In [54]:
#time range
rider['Rid Check Date'].unique()


array(['2020-05-01T00:00:00.000000000', '2020-05-02T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [27]:
rider.columns


Index(['Stop Id', 'Stop Number', 'ETime', 'Actual Arrive Time',
       'Actual Depart Time', 'Sched Arrive Time', 'Sched Depart Time',
       'Sequence', 'Rid Board Count', 'Rid Alight Count', 'Rid Check Type',
       'Line', 'Block Name', 'Bus Number', 'Service Id', 'Rid Check Date',
       'Pattern', 'Pattern Id', 'Trip Id', 'Lat', 'Lon', 'Stop Abbr',
       'Stop Name'],
      dtype='object')

In [55]:
#overall change
rider.groupby(['Rid Check Date'])['Rid Board Count','Rid Alight Count'].sum()


,Rid Board Count,Rid Alight Count
Rid Check Date,,
2020-05-01,6944,6944
2020-05-02,5741,5741


In [60]:
pattern_ag=rider.groupby(['Pattern Id','Rid Check Date'])['Rid Board Count','Rid Alight Count'].sum().reset_index(level=['Rid Check Date'])
pattern_ag.head()

,Rid Check Date,Rid Board Count,Rid Alight Count
Pattern Id,,,
14549,2020-05-01,73,66
14549,2020-05-02,71,48
14550,2020-05-01,51,37
14550,2020-05-02,38,33
14552,2020-05-01,39,46


In [63]:
pattern_ag.pivot_table( values=['Rid Board Count','Rid Alight Count'],index=['Pattern Id'],
                    columns=['Rid Check Date'])

Rid Alight Count            Rid Board Count           
Rid Check Date       2020-05-01 2020-05-02      2020-05-01 2020-05-02
Pattern Id                                                           
14549                      66.0       48.0            73.0       71.0
14550                      37.0       33.0            51.0       38.0
14552                      46.0       64.0            39.0       41.0
14553                      64.0       23.0            50.0       18.0
14554                     127.0       83.0           127.0       80.0
14555                     146.0      111.0           146.0      111.0
14556                      30.0       40.0            39.0       50.0
14557                      28.0        NaN            32.0        NaN
14558                      14.0        5.0            20.0       19.0
14560                       0.0        0.0             0.0        3.0
14561                      42.0        8.0            36.0        4.0
14562                      20.0       41.0             7.0       18.0
14566                     207.0      104.0           139.0       85.0
14568                     153.0      156.0           221.0      175.0
14574                      38.0       73.0            32.0       64.0
14575                      90.0      146.0            73.0      127.0
14576                      74.0       61.0            52.0       40.0
14578                      14.0       19.0            14.0       19.0
14580                      72.0       69.0            94.0       90.0
14581                      10.0       47.0            16.0       56.0
14582                       3.0        4.0             3.0        4.0
14584                      13.0       11.0            18.0       12.0
14585                      56.0      138.0            68.0      156.0
14587                       NaN        1.0             NaN        1.0
14588                      25.0       17.0            28.0       21.0
14589                     111.0      175.0           135.0      185.0
14592                      14.0       25.0            11.0       21.0
14593                     115.0      175.0            91.0      165.0
14607                      61.0        2.0            32.0        2.0
14608                      50.0        NaN            79.0        NaN
...                         ...        ...             ...        ...
14644                     187.0        NaN           116.0        NaN
14646                      54.0        NaN           125.0        NaN
14657                      72.0       48.0            85.0       68.0
14658                      90.0       69.0            97.0       80.0
14659                     138.0       83.0           124.0       58.0
14660                      93.0       59.0            87.0       53.0
14663                     445.0      448.0           445.0      448.0
14668                     494.0      451.0           494.0      451.0
14669                     212.0      162.0           231.0      184.0
14670                     252.0      239.0           287.0      257.0
14671                       8.0        4.0             8.0        4.0
14672                     206.0      183.0           187.0      161.0
14673                     297.0      243.0           262.0      225.0
14677                      45.0      271.0            45.0      271.0
14680                     221.0      341.0           221.0      341.0
14683                     373.0       93.0           426.0      122.0
14686                     309.0      123.0           256.0       94.0
14693                     114.0       99.0           114.0       99.0
14696                     107.0       94.0           107.0       94.0
14699                     138.0       50.0           173.0       75.0
14702                     153.0       72.0           118.0       47.0
14715                       NaN      118.0             NaN      118.0
14717                       NaN      108.0             NaN      108.0
14722                    